In [13]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [14]:
# Load dataset from the local path
data_path = "IMDB Dataset.csv"  # Replace with actual path
df = pd.read_csv(data_path)

# Convert labels to numerical format
df['label'] = df['label'].map({'positive': 1, 'negative': 0})

# Split into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [19]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# Tokenize the input data
def tokenize_data(data):
    return tokenizer(data.tolist(), padding=True, truncation=True, max_length=512, return_tensors='tf')

X_train_enc = tokenize_data(train_df['review'])
X_test_enc = tokenize_data(test_df['review'])

In [25]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train_enc['input_ids'], train_df['label'], 
                    validation_data=(X_test_enc['input_ids'], test_df['label']),
                    epochs=3, batch_size=32)

Epoch 1/3


AttributeError: in user code:

    File "C:\Users\ahmed\anaconda3\envs\neuralnetwork\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\ahmed\anaconda3\envs\neuralnetwork\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ahmed\anaconda3\envs\neuralnetwork\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\ahmed\anaconda3\envs\neuralnetwork\lib\site-packages\transformers\modeling_tf_utils.py", line 1630, in train_step
        x, y, sample_weight = keras.utils.unpack_x_y_sample_weight(data)

    AttributeError: module 'keras.utils' has no attribute 'unpack_x_y_sample_weight'


In [ ]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test_enc['input_ids'], test_df['label'])
print(f'Test Accuracy: {test_acc}')

# Predictions and Classification Report
y_pred_logits = model.predict(X_test_enc['input_ids']).logits
y_pred = np.argmax(y_pred_logits, axis=1)
print(classification_report(test_df['label'], y_pred, target_names=['negative', 'positive']))

In [ ]:
# Function to classify an external review
def classify_review(review):
    inputs = tokenizer(review, padding=True, truncation=True, max_length=512, return_tensors='tf')
    prediction = model.predict(inputs['input_ids']).logits
    sentiment = 'positive' if np.argmax(prediction) == 1 else 'negative'
    print(f'Review: {review}')
    print(f'Predicted Sentiment: {sentiment}')

# Example usage
external_review = "The movie was absolutely fantastic! A must-watch."
classify_review(external_review)